In [ ]:
#langsmith
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
#chat model
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
#embeddings
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [8]:
#vertor store
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [12]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    # web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    web_paths=("https://defensoria.mg.def.br/defensoria-publica-faz-atendimento-presencial-e-remoto-para-pessoas-interessadas-em-aderir-ao-programa-de-indenizacao-referente-ao-rompimento-da-barragem-de-fundao/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [13]:
response = graph.invoke({"question": "Como fazer"})
print(response["answer"])

Para fazer a inscrição, você deve seguir as instruções fornecidas na documentação ou na plataforma específica que está utilizando. Geralmente, isso inclui preencher um formulário com informações pessoais e enviar a solicitação. Se precisar de detalhes específicos sobre o processo, consulte a plataforma ou site correspondente.


In [23]:
#Loading documents
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://defensoria.mg.def.br/defensoria-publica-faz-atendimento-presencial-e-remoto-para-pessoas-interessadas-em-aderir-ao-programa-de-indenizacao-referente-ao-rompimento-da-barragem-de-fundao/",),
    # bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")


Total characters: 10478


In [26]:
print(docs[0].page_content[:15])








  DPMG f


In [28]:
#Splitting documents
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 14 sub-documents.


In [29]:
#store documents
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['4f87f3a2-8e80-45a7-b8da-aa7eaabf4249', 'e8945239-aeab-4c3d-9af4-81eef7a4148f', 'aea4b257-144a-4887-8d74-d6c6e5e47ed7']


In [ ]:
#Retrieval and Generation
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [31]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [ ]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [33]:
#Control flow
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [34]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [35]:
result = graph.invoke({"question": "Em quais cidades?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='977e7405-40fc-4070-b9a2-c128d4bb5dde', metadata={'source': 'https://defensoria.mg.def.br/defensoria-publica-faz-atendimento-presencial-e-remoto-para-pessoas-interessadas-em-aderir-ao-programa-de-indenizacao-referente-ao-rompimento-da-barragem-de-fundao/', 'title': '  DPMG faz atendimento presencial e remoto para interessados em aderir ao programa de indenização referente ao rompimento da barragem de Fundão\xa0\xa0', 'language': 'pt-br', 'start_index': 4221}, page_content='Mutirão de Atendimento\xa0\nPara ampliar o acesso, a Defensoria Pública programou o Mutirão de Atendimento para orientações sobre o acordo de repactuação e o direito de adesão ao PID.\xa0\xa0\nSerão feitos atendimentos itinerantes nos seguintes municípios: Periquito, Alpercata, Barra Longa Rio Doce, Santa Cruz do Escalvado, Sem-Peixe, Rio Casca, São Pedro dos Ferros, Bom Jesus do Galho, Córrego Novo, Pingo D’Água, Conselheiro Pena, Tumiritinga, São Domingos do Prata, Dionísio, São José do Goiaba

In [36]:
for step in graph.stream(
    {"question": "What is Task Decomposition?"}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

{'retrieve': {'context': [Document(id='71952cd5-db8c-4731-80f5-d86804d8533a', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), Document(id='acbb7fa9-695a-4cc1-bf5c-9a422a6d2993', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reaso

In [37]:
for message, metadata in graph.stream(
    {"question": "What is Task Decomposition?"}, stream_mode="messages"
):
    print(message.content, end="|")

|Task| decomposition| is| the| process| of| breaking| down| a| complex| task| into| smaller|,| manageable| steps|.| It| is| often| facilitated| by| techniques| such| as| chain| of| thought| (|Co|T|),| which| encourages| a| model| to| think| step|-by|-step|,| and| the| Tree| of| Thoughts| method|,| which| explores| multiple| reasoning| possibilities| at| each| step|.| This| approach| aids| in| planning| and| enhances| the| model|'s| ability| to| handle| intricate| tasks|.||

In [ ]:
# result = await graph.ainvoke('Como fazer inscrição?')

InvalidUpdateError: Expected dict, got Como fazer inscrição?
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [40]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [41]:
total_documents = len(all_splits)
third = total_documents // 3

for i, document in enumerate(all_splits):
    if i < third:
        document.metadata["section"] = "beginning"
    elif i < 2 * third:
        document.metadata["section"] = "middle"
    else:
        document.metadata["section"] = "end"


all_splits[0].metadata

{'source': 'https://defensoria.mg.def.br/defensoria-publica-faz-atendimento-presencial-e-remoto-para-pessoas-interessadas-em-aderir-ao-programa-de-indenizacao-referente-ao-rompimento-da-barragem-de-fundao/',
 'title': '  DPMG faz atendimento presencial e remoto para interessados em aderir ao programa de indenização referente ao rompimento da barragem de Fundão\xa0\xa0',
 'language': 'pt-br',
 'start_index': 9,
 'section': 'beginning'}

In [42]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(all_splits)